In [16]:
import pandas as pd
import datetime

In [17]:
data = pd.read_excel('Клиент_СПУ.xlsx')

In [18]:
def make_date(x):
    try:
        x = x.date()
        return x
    except:
        return x

In [19]:
data['Дата начала СПУ из карточки Пациента'] = data['Дата начала СПУ из карточки Пациента'].apply(make_date)
data['Дата окончания СПУ из карточки Пациента'] = data['Дата окончания СПУ из карточки Пациента'].apply(make_date)

In [20]:
data = data.sort_values(by=['Номер карты', 'Дата начала СПУ из карточки Пациента'])

In [21]:
group_data = data.groupby('Номер карты', as_index=False).agg({'СПУ из карточки Пациента':list, 'Дата начала СПУ из карточки Пациента':list, 'Дата окончания СПУ из карточки Пациента':list}).reset_index(drop=True)

In [22]:
group_data[group_data['Номер карты'] == '540029723']

,Номер карты,СПУ из карточки Пациента,Дата начала СПУ из карточки Пациента,Дата окончания СПУ из карточки Пациента
27720,540029723,"[Базовая 0-1 стар, Скидка 5%, Скидка 5%, Скидк...","[2017-02-15, 2017-12-02, 2017-12-03, 2020-01-30]","[2017-12-31, 2020-01-29, 2019-12-13, -]"


In [24]:
guide_dict = {}

for i in group_data.index:
    guide_dict.setdefault(group_data['Номер карты'][i], {})
    for j in range(len(group_data['СПУ из карточки Пациента'][i])):
        guide_dict[group_data['Номер карты'][i]].setdefault(group_data['СПУ из карточки Пациента'][i][j], \
                                                           (group_data['Дата начала СПУ из карточки Пациента'][i][j], group_data['Дата окончания СПУ из карточки Пациента'][i][j]))
    

In [26]:
guide_dict['540029723']

{'Базовая 0-1 стар': (datetime.date(2017, 2, 15), datetime.date(2017, 12, 31)),
 'Скидка 5%': (datetime.date(2017, 12, 2), datetime.date(2020, 1, 29)),
 'Скидка 15%': (datetime.date(2020, 1, 30), '-')}

In [61]:
data = pd.read_excel('Данные_НСК.xlsx')

In [64]:
data['Дата приема'] = data['Дата приема'].apply(make_date)

In [62]:
data.head()

,Номер карты,Пациент,Пол,Дата рождения,Категория клиента,Год,Месяц,Дата приема,Дата и время посещения,Дата первого посещения,Наименование услуги,Sum([Количество]),Sum([Всего начислено]),Электронный адрес Пациента
0,540025226,Андронов Тимофей Аркадьевич,Мужской,2015-09-14 00:00:00,НАЛ,2017,янв.,2017-01-02,2017-01-02 15:33:13,2016-01-23,УЗИ сердца (эхокардиография) в амбулатории,1,1500.0,-
1,540031560,Афонина Оксана Юрьевна,Женский,1982-12-20 00:00:00,НАЛ,2017,янв.,2017-01-02,2017-01-02 13:46:42,2017-01-02,"Консультация врача - гинеколога, зав.отд., вед...",1,1200.0,-
2,540031560,Афонина Оксана Юрьевна,Женский,1982-12-20 00:00:00,НАЛ,2017,янв.,2017-01-02,2017-01-02 13:20:31,2017-01-02,Консультация врача - отоларинголога первичная ...,1,900.0,-
3,540031560,Афонина Оксана Юрьевна,Женский,1982-12-20 00:00:00,НАЛ,2017,янв.,2017-01-02,2017-01-02 16:04:26,2017-01-02,УЗИ органов малого таза комплексное (гинеколог...,1,1400.0,-
4,540031552,Байгужинов Александр Сергеевич,Мужской,1990-02-28 00:00:00,НАЛ,2017,янв.,2017-01-02,2017-01-02 09:59:45,2017-01-02,Консультация врача - терапевта первичная в амб...,1,900.0,-


In [102]:
temp_arr = []
new_temp_arr = []
for i in data.index:
    temp_arr.append('')
    new_temp_arr.append('')
    flag = False
    temp_date = data['Дата приема'][i]
    if len(guide_dict[data['Номер карты'][i]]) == 1 and '-' in guide_dict[data['Номер карты'][i]].keys():
        temp_arr[-1] = '-'
        new_temp_arr[-1] = '-'
        continue
    else:
        for key, val in guide_dict[data['Номер карты'][i]].items():
            if flag:
                break
            if temp_date >= val[0]:
                if val[1] == '-' or temp_date <= val[1]:
                    temp_arr[-1] = key
                    new_temp_arr[-1] = val
                    flag = True
    if temp_arr[-1] == '':
        temp_arr[-1] = '-'
        new_temp_arr[-1] = '-'

            

In [103]:
data['Действующая СПУ на дату приема'] = temp_arr
data['Сроки действия СПУ'] = new_temp_arr

In [104]:
data.to_excel('Обработано.xlsx')